## Integrantes do grupo:

**(Nome do aluno) - (R.A)**

Luiz Fernando Ferreira Berber - 817123421

Lucas Alves de Sousa - 817116864

Matheus Henrique Silva Paiva - 817123241 

Mateus Sampaio de Souza - 817123420

Pedro Jacó Cunha Mesquita - 817124121

In [1]:
!pip install numpy
!pip install pandas
!pip install regex
!pip install mysql-connector-python

In [2]:
import numpy as np
import pandas as pd
import mysql.connector
import regex

from mysql.connector import Error
from pathlib import Path

In [3]:
def get_files_from_path(filepath):
    path = Path(filepath)
    files = []

    for childs in path.iterdir():
        if childs.is_file(): files.append(childs.parts[len(childs.parts) - 1])

    return files

In [4]:
def create_connection(host_name, user_name, user_password):
    connection = None
    
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [5]:
def execute_par_query(connection, query, pars):
    cursor = connection.cursor()
    try:
        cursor.execute(query, pars)
        connection.commit()
        
        if query.lstrip().upper().startswith('INSERT'):
            return cursor.lastrowid
        else:
            return 0
    except Error as e:
        print(f"The error '{e}' occurred")
        print(f'The error query: {query}')

In [6]:
def criar_dataframe_area_atuacao(df_orcamento_despesa):
    return pd.DataFrame({
        'database_id':df_orcamento_despesa['CÓDIGO FUNÇÃO'],
        'cod_funcao':df_orcamento_despesa['CÓDIGO FUNÇÃO'],
        'nome_funcao':df_orcamento_despesa['NOME FUNÇÃO'],
        'cod_sub_funcao':df_orcamento_despesa['CÓDIGO SUBFUNÇÃO'],
        'nome_sub_funcao':df_orcamento_despesa['NOME SUBFUNÇÃO'],
        'valor_orcado':df_orcamento_despesa['ORÇAMENTO INICIAL (R$)']
    })

In [7]:
def criar_dataframe_orgao(df_despesa):
    return pd.DataFrame({
        'database_id':df_despesa['Código Órgão Superior'],
        'cod_orgao_superior':df_despesa['Código Órgão Superior'],
        'nome_orgao_superior':df_despesa['Nome Órgão Superior'],
        'cod_orgao_subordinado':df_despesa['Código Órgão Subordinado'],
        'nome_orgao_subordinado':df_despesa['Nome Órgão Subordinado'],
        'cod_unidade_orcamentaria':df_despesa['Código Unidade Orçamentária'],
        'nome_unidade_orcamentaria':df_despesa['Nome Unidade Orçamentária'],
        'cod_funcao':df_despesa['Código Função'],
        'nome_funcao':df_despesa['Nome Função'],
        'cod_sub_funcao':df_despesa['Código Subfução'],
        'nome_sub_funcao':df_despesa['Nome Subfunção']
    })

In [8]:
def criar_dataframe_programa(df_despesa):
    return pd.DataFrame({
        'database_id':df_despesa['Código Programa Orçamentário'],
        'cod_programa_orcamentario':df_despesa['Código Programa Orçamentário'],
        'nome_programa_orcamentario':df_despesa['Nome Programa Orçamentário'],
        'cod_acao':df_despesa['Código Ação'],
        'nome_acao':df_despesa['Nome Ação'],
        'cod_funcao':df_despesa['Código Função'],
        'nome_funcao':df_despesa['Nome Função'],
        'cod_sub_funcao':df_despesa['Código Subfução'],
        'nome_sub_funcao':df_despesa['Nome Subfunção'],
        'valor_liquidado':df_despesa['Valor Liquidado (R$)']
    })

In [9]:
def criar_dataframe_data(nome_arquivo):
    match = regex.match('(?P<ano>\d{4})(?P<mes>\d{2})', nome_arquivo)
    ano = match.group('ano')
    mes = match.group('mes')
    
    return pd.DataFrame({
        'database_id': [mes],
        'mes':[mes],
        'ano':[ano]
    })

In [10]:
def criar_dataframe_fato():
    return pd.DataFrame({
        'id_dimensao_data': [],
        'id_dimensao_orgao': [],
        'id_dimensao_programa': [],
        'id_dimensao_area_atuacao': [],
        'valor_orcado': [],
        'valor_liquidado': []
    })

In [11]:
def insert_dimensao_area_atuacao(df_dimensao_area_atuacao, connection):
    insert_query = """
    INSERT INTO ORCAMENTO_PUBLICO.TD_AREA_ATUACAO(COD_FUNCAO, NOME_FUNCAO, COD_SUB_FUNCAO, NOME_SUB_FUNCAO, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, NOW());"""
    
    for i in range(df_dimensao_area_atuacao.shape[0]):
        if int(df_dimensao_area_atuacao.loc[i]['cod_funcao']) < 0:
            df_dimensao_area_atuacao.at[i, 'cod_funcao'] = 0
        elif int(df_dimensao_area_atuacao.loc[i]['cod_funcao']) == 0:
            df_dimensao_area_atuacao.at[i, 'cod_funcao'] = 1
            
        query_param = (int(df_dimensao_area_atuacao.loc[i]['cod_funcao']), df_dimensao_area_atuacao.loc[i]['nome_funcao'], int(df_dimensao_area_atuacao.loc[i]['cod_sub_funcao']), df_dimensao_area_atuacao.loc[i]['nome_sub_funcao'])
        inserted_id = execute_par_query(connection, insert_query, query_param)
        df_dimensao_area_atuacao.at[i, 'database_id'] = inserted_id

In [12]:
def insert_dimensao_data(df_dimensao_data, connection):
    insert_query = """
    INSERT INTO ORCAMENTO_PUBLICO.TD_DATA(MES, ANO, DATA_CRIACAO)
    VALUES(%s, %s, NOW());"""
    
    query_param = (int(df_dimensao_data['mes'][0]), int(df_dimensao_data['ano'][0]))
    inserted_id = execute_par_query(connection, insert_query, query_param)
    
    df_dimensao_data.at[0, 'database_id'] = inserted_id

In [13]:
def insert_dimensao_orgao(df_dimensao_orgao, connection):    
    insert_query = """
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());"""

    for i in range(df_dimensao_orgao.shape[0]):
        if int(df_dimensao_orgao.loc[i]['cod_orgao_superior']) < 0:
            df_dimensao_orgao.at[i, 'cod_orgao_superior'] = 0
        elif int(df_dimensao_orgao.loc[i]['cod_orgao_superior']) == 0:
            df_dimensao_orgao.at[i, 'cod_orgao_superior'] = 1
            
        query_param = (int(df_dimensao_orgao.loc[i]['cod_orgao_superior']), df_dimensao_orgao.loc[i]['nome_orgao_superior'], int(df_dimensao_orgao.loc[i]['cod_orgao_subordinado']), df_dimensao_orgao.loc[i]['nome_orgao_subordinado'], int(df_dimensao_orgao.loc[i]['cod_unidade_orcamentaria']), df_dimensao_orgao.loc[i]['nome_unidade_orcamentaria'])
        inserted_id = execute_par_query(connection, insert_query, query_param)
        df_dimensao_orgao.at[i, 'database_id'] = inserted_id

In [14]:
def insert_dimensao_programa(df_dimensao_programa, connection):
    insert_query = """
    INSERT INTO ORCAMENTO_PUBLICO.TD_PROGRAMA(Cod_Programa_Orcamentario, Nome_Programa_Orcamentario, Cod_Acao, Nome_Acao, Data_Criacao)
    VALUES(%s, %s, %s, %s, NOW());"""
    
    for i in range(df_dimensao_programa.shape[0]):
        if int(df_dimensao_programa.loc[i]['cod_programa_orcamentario']) < 0:
            df_dimensao_programa.at[i, 'cod_programa_orcamentario'] = 0
        elif int(df_dimensao_programa.loc[i]['cod_programa_orcamentario']) == 0:
            df_dimensao_programa.at[i, 'cod_programa_orcamentario'] = 1
            
        query_param = (int(df_dimensao_programa.loc[i]['cod_programa_orcamentario']), df_dimensao_programa.loc[i]['nome_programa_orcamentario'], df_dimensao_programa.loc[i]['cod_acao'], df_dimensao_programa.loc[i]['nome_acao'])        
        inserted_id = execute_par_query(connection, insert_query, query_param)
        df_dimensao_programa.at[i, 'database_id'] = inserted_id

In [52]:
def unir_dataframes(df_dimensao_data, df_dimensao_orgao, df_dimensao_programa):
    database_id_data = int(df_dimensao_data['database_id'][0])

    df = df_dimensao_orgao.join(df_dimensao_programa, lsuffix='_orgao', rsuffix='_programa')

    df['database_id_data'] = pd.Series(np.random.randn(len(df['cod_funcao_orgao'])), index=df.index)

    for row in range(df.shape[0]):
        df.at[row, 'database_id_data'] = database_id_data

    return df

In [184]:
def unir_dataframe_fato(df_fato, df_dimensao_area_atuacao, df_dimensoes):
    for row in range(df_dimensao_area_atuacao.shape[0]):
        cod_funcao = int(df_dimensao_area_atuacao.loc[row]['cod_funcao'])
        cod_sub_funcao = int(df_dimensao_area_atuacao.loc[row]['cod_sub_funcao'])
        cod_programa_orcamentario = int(df_dimensoes.loc[row]['cod_programa_orcamentario'])
        
        valor = float(df_dimensao_area_atuacao.loc[0]['valor_orcado'].replace(',', '.'))
        valor_orcado = 0.0 if valor == 0 else valor / 12

        df_dimensoes_por_funcao = df_dimensoes.query(f'cod_funcao_orgao == {cod_funcao} & cod_sub_funcao_orgao == {cod_sub_funcao}')
        
        df_dimensoes_por_funcao['database_id_area_atuacao'] = pd.Series(np.random.randn(len(df_dimensoes_por_funcao['cod_funcao_orgao'])), index=df_dimensoes_por_funcao.index)
        df_dimensoes_por_funcao['valor_orcado'] = pd.Series(np.random.randn(len(df_dimensoes_por_funcao['cod_funcao_orgao'])), index=df_dimensoes_por_funcao.index)
        
        for row_df in range(df_dimensoes_por_funcao.shape[0]):
            df_dimensoes_por_funcao.iat[row_df, 22] = df_dimensao_area_atuacao.loc[row]['database_id']
            df_dimensoes_por_funcao.iat[row_df, 23] = valor_orcado
            
            df_fato.append({
                'id_dimensao_data':df_dimensoes_por_funcao['database_id_data'],
                'id_dimensao_orgao':df_dimensoes_por_funcao['database_id_orgao'],
                'id_dimensao_programa':df_dimensoes_por_funcao['database_id_programa'],
                'id_dimensao_area_atuacao':df_dimensoes_por_funcao['database_id_area_atuacao'],
                'valor_orcado':df_dimensoes_por_funcao['valor_orcado'],
                'valor_liquidado':df_dimensoes_por_funcao['valor_liquidado']
            }, ignore_index=True)

In [17]:
mysql_host_name = '127.0.0.1'
mysql_user_name = 'mateus'
mysql_password = 'root123'

In [18]:
path_dados = '../Datas'

df_fato = criar_dataframe_fato()

In [19]:
connection = create_connection(mysql_host_name, mysql_user_name, mysql_password)

Connection to MySQL DB successful


In [20]:
path_orcamento_despesas = f'{path_dados}/OrcamentoDespesas'

arquivos = get_files_from_path(path_orcamento_despesas)

df_orcamento_despesas = pd.read_csv(f'{path_orcamento_despesas}/{arquivos[0]}', sep=';', encoding='mbcs')

In [21]:
df_dimensao_area_atuacao = criar_dataframe_area_atuacao(df_orcamento_despesas)

In [22]:
insert_dimensao_area_atuacao(df_dimensao_area_atuacao, connection)

In [23]:
path_despesa = f'{path_dados}/Despesas'

arquivos = get_files_from_path(path_despesa)

In [25]:
for arquivo in arquivos:
    print(arquivo)
    
    dados_despesa = pd.read_csv(f'{path_despesa}/{arquivo}', sep=';', encoding='mbcs')
    
    df_dimensao_data = criar_dataframe_data(arquivo)
    df_dimensao_orgao = criar_dataframe_orgao(dados_despesa)
    df_dimensao_programa = criar_dataframe_programa(dados_despesa)
    
    print('Data Frames Criados')
    print('Importando...')
    
    insert_dimensao_data(df_dimensao_data, connection)
    print('Importação dimensão data concluída')
    insert_dimensao_orgao(df_dimensao_orgao, connection)
    print('Importação dimensão orgao concluída')
    insert_dimensao_programa(df_dimensao_programa, connection)
    print('Importação dimensão programa concluída')
    print('Importações Concluídas')
    
    df_dimensoes = unir_dataframes(df_dimensao_data, df_dimensao_orgao, df_dimensao_programa)
    print('Unindo dataframes dimensoes')
    
    unir_dataframe_fato(df_fato, df_dimensao_area_atuacao, df_dimensoes)
    break

201901_Despesas.zip
Data Frames Criados
Importando...
Importação dimensão data concluída
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NO

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

...

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

KeyError: 'cod_programa_orcamentario'

In [54]:
df_dimensoes = unir_dataframes(df_dimensao_data, df_dimensao_orgao, df_dimensao_programa)

df_dimensoes

,database_id_orgao,cod_orgao_superior,nome_orgao_superior,cod_orgao_subordinado,nome_orgao_subordinado,cod_unidade_orcamentaria,nome_unidade_orcamentaria,cod_funcao_orgao,nome_funcao_orgao,cod_sub_funcao_orgao,...,cod_programa_orcamentario,nome_programa_orcamentario,cod_acao,nome_acao,cod_funcao_programa,nome_funcao_programa,cod_sub_funcao_programa,nome_sub_funcao_programa,valor_liquidado,database_id_data
0,790774.0,33000,Ministério da Previdência Social,37904,Fundo do Regime Geral da Previdência Social,55902,FUNDO DO REGIME GERAL DA PREVID.SOCIAL-FRGPS,9,Previdência social,271,...,2061,PREVIDENCIA SOCIAL,0E81,BENEFICIOS PREVIDENCIARIOS URBANOS,9,Previdência social,271,Previdência básica,"23518,5900",28.0
1,790775.0,26000,Ministério da Educação,26406,Instituto Federal do Espírito Santo,26406,"INST.FED.DE EDUC.,CIENC.E TEC.DO ESP.SANTO",12,Educação,363,...,2080,EDUCACAO DE QUALIDADE PARA TODOS,20RL,FUNCIONAMENTO DAS INSTITUICOES DA REDE FEDERAL...,12,Educação,363,Ensino profissional,"0,0000",28.0
2,790776.0,30000,Ministério da Justiça e Segurança Pública,30000,Ministério da Justiça e Segurança Pública - Un...,82101,MINISTERIO DA SEGURANCA PUBLICA,27,Desporto e lazer,812,...,180,ESPORTE SOLIDARIO,5450,IMPLANTACAO E MODERNIZACAO DE INFRAESTRUTURA P...,27,Desporto e lazer,812,Desporto comunitário,"297202,6700",28.0
3,790777.0,24000,"Ministério da Ciência, Tecnologia, Inovações",24000,"Ministério da Ciência, Tecnologia, Inovações e...",24101,"MINIST.DA CIENCIA,TECNOL.,INOV.E COMUNICACOES",19,Ciência e Tecnologia,122,...,2106,PROGRAMA DE GESTAO E MANUTENCAO DO MINISTERIO ...,2000,ADMINISTRACAO DA UNIDADE,19,Ciência e Tecnologia,122,Administração geral,"0,0000",28.0
4,790778.0,30000,Ministério da Justiça e Segurança Pública,30108,Departamento de Polícia Federal,82102,DEPARTAMENTO DE POLICIA FEDERAL,6,Segurança pública,122,...,2135,PROGRAMA DE GESTAO E MANUTENCAO DO MINISTERIO ...,2000,ADMINISTRACAO DA UNIDADE,6,Segurança pública,122,Administração geral,"7922,4300",28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42756,833184.0,26000,Ministério da Educação,26407,Instituto Federal Goiano,26407,"INST.FED.DE EDUC.,CIENC.E TEC.GOIANO",12,Educação,363,...,2080,EDUCACAO DE QUALIDADE PARA TODOS,20RL,FUNCIONAMENTO DAS INSTITUICOES DA REDE FEDERAL...,12,Educação,363,Ensino profissional,"79,4800",28.0
42757,833185.0,35000,Ministério das Relações Exteriores,35000,Ministério das Relações Exteriores - Unidades ...,35101,MINISTERIO DAS RELACOES EXTERIORES,7,Relações exteriores,211,...,2082,POLITICA EXTERNA,20WW,RELACOES E NEGOCIACOES BILATERAIS,7,Relações exteriores,211,Relações diplomáticas,"0,0000",28.0
42758,833186.0,52000,Ministério da Defesa,52121,Comando do Exército,52121,COMANDO DO EXERCITO,5,Defesa nacional,122,...,2108,PROGRAMA DE GESTAO E MANUTENCAO DO MINISTERIO ...,2000,ADMINISTRACAO DA UNIDADE,5,Defesa nacional,122,Administração geral,"0,0000",28.0
42759,833187.0,22000,"Ministério da Agricultura, Pecuária e Abastec",22202,Empresa Brasileira de Pesquisa Agropecuária,22202,EMPRESA BRASILEIRA DE PESQUISA AGROPECUARIA,20,Agricultura,572,...,2042,PESQUISA E INOVACOES PARA A AGROPECUARIA,20Y6,PESQUISA E DESENVOLVIMENTO DE TECNOLOGIAS PARA...,20,Agricultura,572,Desenvolvimento tecnológico e engenharia,"0,0000",28.0


In [ ]:
unir_dataframe_fato(df_fato, df_dimensao_area_atuacao, df_dimensoes)

<ipython-input-184-19a5ffb40b6f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dimensoes_por_funcao['database_id_area_atuacao'] = pd.Series(np.random.randn(len(df_dimensoes_por_funcao['cod_funcao_orgao'])), index=df_dimensoes_por_funcao.index)
<ipython-input-184-19a5ffb40b6f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dimensoes_por_funcao['valor_orcado'] = pd.Series(np.random.randn(len(df_dimensoes_por_funcao['cod_funcao_orgao'])), index=df_dimensoes_por_funcao.index)


In [ ]:
df_fato